In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from skl2onnx.helpers.onnx_helper import select_model_inputs_outputs
from skl2onnx.helpers.onnx_helper import save_onnx_model
from skl2onnx.helpers.onnx_helper import enumerate_model_node_outputs
from skl2onnx.helpers.onnx_helper import load_onnx_model

from stitchnet.stitchonnx.utils import change_input_dim, get_fragments, Net

import numpy as np
from tqdm import tqdm
from glob import glob
from stitchnet.finetuning.finetune import finetune
import os
import torch

num_classes = 2
num_epochs = 50
feature_extract = True
batch_size = 64

modelnames = sorted(glob('_models/*.onnx'))

os.makedirs('_results/finetune', exist_ok=True)
for i,modelname in tqdm(enumerate(modelnames)):
    model_onnx1 = load_onnx_model(modelname)
    model_ft, hist = finetune(model_onnx1, num_classes, num_epochs, batch_size, feature_extract)
    name = os.path.basename(modelname)
    namewithoutext = os.path.splitext(name)[0]
    torch.onnx.export(model_ft, torch.ones(1,3,224,224), f'_results/finetune/{namewithoutext}_ft.onnx')
    with open(f'_results/finetune/{namewithoutext}.txt', 'w') as f:
        for line in hist:
            f.write(f'{",".join([f"{x:.4}" for x in line])}\n')    

0it [00:00, ?it/s]

Params to learn:
	 Gemm_2.weight
	 Gemm_2.bias
Epoch 0/49
----------
train Loss: 0.6297 Acc: 0.6543
